# GCP-Based Patch Matching for Orthomosaic Registration

This notebook performs patch matching to find ground control points (GCPs) in orthomosaics and uses them to register the orthos to the basemap.

## Approach:
1. Load GCPs from CSV file (WGS84 coordinates)
2. Extract patches from basemap centered on each GCP
3. Use template matching to find corresponding patches in orthomosaics
4. Compute 2D shift or affine transformation from matches
5. Apply transformation to register orthos to basemap
6. Evaluate accuracy improvement

## Inputs:
- **Basemap**: `TestsiteNewWest_Spexigeo_RTK.tiff`
- **GCPs**: `25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv` (converted to WGS84)
- **Orthomosaics**: 
  - `outputs/orthomosaics/orthomosaic_no_gcps.tif`
  - `outputs/orthomosaics/orthomosaic_with_gcps.tif`

## Outputs:
- All outputs saved to `outputs/gcp_matching/`
- Patches extracted from basemap
- Matched GCP locations in orthos
- Registered orthomosaics
- Accuracy evaluation

## Setup: Install Dependencies

In [ ]:
# Install required packages if needed
import sys
import subprocess

packages = [
    'rasterio',
    'numpy',
    'matplotlib',
    'opencv-python',
    'scipy',
    'utm',
    'pillow'
]

for package in packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

print("✓ Dependencies installed")

## Step 1: Setup - Imports and Paths

In [2]:
import numpy as np
import rasterio
from rasterio.transform import xy
from rasterio.warp import transform as transform_coords
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
import json
import csv
import utm
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Setup paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")

# Input files
basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
gcp_csv_path = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
ortho_no_gcps_path = output_dir / "orthomosaics" / "orthomosaic_no_gcps.tif"
ortho_with_gcps_path = output_dir / "orthomosaics" / "orthomosaic_with_gcps.tif"

# Output directories
gcp_matching_dir = output_dir / "gcp_matching"
gcp_matching_dir.mkdir(parents=True, exist_ok=True)

patches_dir = gcp_matching_dir / "patches"
patches_dir.mkdir(exist_ok=True)

matches_dir = gcp_matching_dir / "matches"
matches_dir.mkdir(exist_ok=True)

registered_dir = gcp_matching_dir / "registered"
registered_dir.mkdir(exist_ok=True)

print(f"✓ Output directory: {gcp_matching_dir}")
print(f"  - Patches: {patches_dir}")
print(f"  - Matches: {matches_dir}")
print(f"  - Registered: {registered_dir}")

✓ Output directory: outputs/gcp_matching
  - Patches: outputs/gcp_matching/patches
  - Matches: outputs/gcp_matching/matches
  - Registered: outputs/gcp_matching/registered


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Update paths for Colab
data_dir = Path("/content/drive/MyDrive/Data/New Westminster Oct _25")
output_dir = Path("/content/drive/MyDrive/Code/MyCode/research-westminster_ground_truth_analysis/outputs")

# Update other paths accordingly
basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
gcp_csv_path = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
ortho_no_gcps_path = output_dir / "orthomosaics" / "orthomosaic_no_gcps.tif"
ortho_with_gcps_path = output_dir / "orthomosaics" / "orthomosaic_with_gcps.tif"

print("✓ Colab paths configured")

## Step 2: Load GCPs from CSV and Convert to WGS84

In [3]:
# Load GCPs - try existing WGS84 files first, otherwise parse CSV
import json

# Check for existing WGS84 GCP files from ground control comparison
gcps_wgs84_geojson = output_dir / "ground_control_comparison" / "gcps_wgs84.geojson"
gcps_wgs84_csv = output_dir / "ground_control_comparison" / "gcps_wgs84.csv"

gcps = []

# Try GeoJSON first (preferred)
if gcps_wgs84_geojson.exists():
    print(f"Loading GCPs from GeoJSON: {gcps_wgs84_geojson}")
    with open(gcps_wgs84_geojson, 'r') as f:
        geojson_data = json.load(f)
    
    if 'features' in geojson_data:
        for feature in geojson_data['features']:
            props = feature.get('properties', {})
            geom = feature.get('geometry', {})
            
            if geom.get('type') == 'Point':
                coords = geom.get('coordinates', [])
                if len(coords) >= 2:
                    lon, lat = coords[0], coords[1]
                    
                    # Get UTM coordinates from properties or convert
                    x_utm = props.get('x_utm')
                    y_utm = props.get('y_utm')
                    
                    if x_utm is None or y_utm is None:
                        # Convert WGS84 to UTM
                        import utm
                        x_utm, y_utm, zone_num, zone_letter = utm.from_latlon(lat, lon)
                    
                    gcps.append({
                        'id': props.get('id', props.get('name', f"GCP_{len(gcps)+1}")),
                        'lat': lat,
                        'lon': lon,
                        'x_utm': float(x_utm),
                        'y_utm': float(y_utm)
                    })
    
    print(f"✓ Loaded {len(gcps)} GCPs from GeoJSON")

# Try CSV if GeoJSON not found
elif gcps_wgs84_csv.exists():
    print(f"Loading GCPs from WGS84 CSV: {gcps_wgs84_csv}")
    with open(gcps_wgs84_csv, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                lat = float(row.get('lat', row.get('latitude', 0)))
                lon = float(row.get('lon', row.get('longitude', row.get('lon', 0))))
                gcp_id = row.get('id', row.get('name', row.get('label', f"GCP_{len(gcps)+1}")))
                
                # Get UTM from row or convert
                x_utm_str = row.get('x_utm', '')
                y_utm_str = row.get('y_utm', '')
                
                if x_utm_str and y_utm_str:
                    x_utm = float(x_utm_str)
                    y_utm = float(y_utm_str)
                else:
                    import utm
                    x_utm, y_utm, zone_num, zone_letter = utm.from_latlon(lat, lon)
                
                gcps.append({
                    'id': gcp_id,
                    'lat': lat,
                    'lon': lon,
                    'x_utm': x_utm,
                    'y_utm': y_utm
                })
            except (ValueError, KeyError) as e:
                print(f"⚠️  Skipping row: {e}")
                continue
    
    print(f"✓ Loaded {len(gcps)} GCPs from WGS84 CSV")

# Fallback to parsing UTM CSV
if len(gcps) == 0:
    print(f"\nNo WGS84 GCP files found, parsing UTM CSV: {gcp_csv_path}")
    gcps = load_gcps_from_csv(gcp_csv_path)
    print(f"✓ Loaded {len(gcps)} GCPs from UTM CSV")

if len(gcps) > 0:
    print(f"\nFirst few GCPs:")
    for gcp in gcps[:3]:
        print(f"  {gcp['id']}: UTM=({gcp['x_utm']:.2f}, {gcp['y_utm']:.2f}), WGS84=({gcp['lat']:.6f}, {gcp['lon']:.6f})")
else:
    print(f"⚠️  No GCPs loaded!")
    print(f"   Checked:")
    print(f"   - {gcps_wgs84_geojson}")
    print(f"   - {gcps_wgs84_csv}")
    print(f"   - {gcp_csv_path}")


⚠️  Could not find required columns. Found: ['1', '5450945.525', '506914.123', '77.453', 'GCP1']
   Looking for: name/id, x/easting, y/northing
✓ Loaded 0 GCPs from CSV
⚠️  No GCPs loaded! Check CSV format.
   CSV path: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv
   File exists. Showing first few lines:
     1,5450945.525,506914.123,77.453,GCP1
     2,5450730.008,506657.794,79.218,GCP2
     3,5450480.009,506577.772,59.4,GCP3
     4,5450578.626,506765.034,65.591,GCP4
     5,5450715.958,506926.13,63.103,GCP5


In [4]:
# Convert GCPs (UTM) to pixel coordinates in basemap
def gcp_to_pixel_coords_from_utm(gcp_x_utm: float, gcp_y_utm: float, raster_path: Path) -> Optional[Tuple[int, int]]:
    """
    Convert GCP UTM coordinates to pixel coordinates in raster.
    
    Args:
        gcp_x_utm: UTM Easting (EPSG:32610)
        gcp_y_utm: UTM Northing (EPSG:32610)
        raster_path: Path to raster file
    
    Returns:
        (col, row) or None if outside bounds.
    """
    with rasterio.open(raster_path) as src:
        # Raster should be in EPSG:32610 (UTM Zone 10N)
        if src.crs != 'EPSG:32610':
            # Transform UTM to raster CRS if needed
            x, y = transform_coords(
                'EPSG:32610',
                src.crs,
                [gcp_x_utm],
                [gcp_y_utm]
            )
            utm_x, utm_y = x[0], y[0]
        else:
            utm_x, utm_y = gcp_x_utm, gcp_y_utm
        
        # Convert to pixel coordinates
        row, col = rasterio.transform.rowcol(src.transform, utm_x, utm_y)
        
        # Check if within bounds
        if 0 <= row < src.height and 0 <= col < src.width:
            return (col, row)
        else:
            return None

# Get basemap CRS and transform
with rasterio.open(basemap_path) as basemap_src:
    basemap_crs = basemap_src.crs
    basemap_transform = basemap_src.transform
    basemap_width = basemap_src.width
    basemap_height = basemap_src.height
    basemap_bounds = basemap_src.bounds

print(f"Basemap CRS: {basemap_crs}")
print(f"Basemap dimensions: {basemap_width}x{basemap_height}")
print(f"Basemap bounds: {basemap_bounds}")
print(f"Basemap transform: {basemap_transform}")

# Convert all GCPs to pixel coordinates
gcp_pixel_coords = {}
for gcp in gcps:
    # Debug: show GCP UTM coordinates
    print(f"\nGCP {gcp['id']}: UTM=({gcp['x_utm']:.2f}, {gcp['y_utm']:.2f})")
    
    pixel_coords = gcp_to_pixel_coords_from_utm(gcp['x_utm'], gcp['y_utm'], basemap_path)
    if pixel_coords:
        gcp_pixel_coords[gcp['id']] = {
            'gcp': gcp,
            'pixel_col': pixel_coords[0],
            'pixel_col': pixel_coords[0],
            'pixel_row': pixel_coords[1]
            'utm_x': gcp.get('x_utm'),
            'utm_y': gcp.get('y_utm'),
            'pixel_row': pixel_coords[1]
        }
        print(f"  ✓ Found at pixel: col={pixel_coords[0]}, row={pixel_coords[1]}")
    else:
        # Debug: show why it's outside bounds
        with rasterio.open(basemap_path) as src:
            row, col = rasterio.transform.rowcol(src.transform, gcp['x_utm'], gcp['y_utm'])
            print(f"  ⚠️  Outside bounds: col={col}, row={row}")
            print(f"     Basemap: {src.width}x{src.height}")
            print(f"     Basemap bounds: {src.bounds}")
            # Check if coordinates are in bounds in UTM space
            in_x = src.bounds.left <= gcp['x_utm'] <= src.bounds.right
            in_y = src.bounds.bottom <= gcp['y_utm'] <= src.bounds.top
            print(f"     UTM X in bounds: {in_x} ({src.bounds.left:.2f} <= {gcp['x_utm']:.2f} <= {src.bounds.right:.2f})")
            print(f"     UTM Y in bounds: {in_y} ({src.bounds.bottom:.2f} <= {gcp['y_utm']:.2f} <= {src.bounds.top:.2f})")

print(f"\n✓ Found {len(gcp_pixel_coords)} GCPs within basemap bounds")
if len(gcp_pixel_coords) > 0:
    print(f"\nFirst few GCP pixel coordinates:")
    for gcp_id, coords in list(gcp_pixel_coords.items())[:3]:
        print(f"  {gcp_id}: col={coords['pixel_col']}, row={coords['pixel_row']}")


Basemap CRS: EPSG:32610
Basemap dimensions: 90129x90188
Basemap bounds: BoundingBox(left=506424.37839793676, bottom=5450017.622213458, right=507501.0951215451, top=5451095.043774429)
Basemap transform: | 0.01, 0.00, 506424.38|
| 0.00,-0.01, 5451095.04|
| 0.00, 0.00, 1.00|

GCP GCP1: UTM=(506914.12, 5450945.53)
  ✓ Found at pixel: col=40995, row=12515

GCP GCP2: UTM=(506657.79, 5450730.01)
  ✓ Found at pixel: col=19538, row=30556

GCP GCP3: UTM=(506577.77, 5450480.01)
  ✓ Found at pixel: col=12840, row=51482

GCP GCP4: UTM=(506765.03, 5450578.63)
  ✓ Found at pixel: col=28515, row=43227

GCP GCP5: UTM=(506926.13, 5450715.96)
  ✓ Found at pixel: col=42000, row=31732

GCP GCP6: UTM=(507071.92, 5450992.66)
  ✓ Found at pixel: col=54203, row=8570

GCP GCP7: UTM=(507089.40, 5450794.23)
  ✓ Found at pixel: col=55667, row=25180

GCP GCP8: UTM=(507315.01, 5450717.85)
  ✓ Found at pixel: col=74551, row=31574

GCP GCP9: UTM=(507252.65, 5450536.03)
  ✓ Found at pixel: col=69332, row=46793

GCP GCP

In [ ]:
# Convert GCPs (WGS84) to pixel coordinates in basemap
def gcp_to_pixel_coords(gcp_lat: float, gcp_lon: float, raster_path: Path) -> Optional[Tuple[int, int]]:
    """
    Convert GCP lat/lon to pixel coordinates in raster.
    
    Returns (col, row) or None if outside bounds.
    """
    with rasterio.open(raster_path) as src:
        # Transform WGS84 to raster CRS
        x, y = transform_coords(
            'EPSG:4326',  # WGS84
            src.crs,
            [gcp_lon],
            [gcp_lat]
        )
        
        # Convert to pixel coordinates
        row, col = rasterio.transform.rowcol(src.transform, x[0], y[0])
        
        # Check if within bounds
        if 0 <= row < src.height and 0 <= col < src.width:
            return (col, row)
        else:
            return None

# Get basemap CRS and transform
with rasterio.open(basemap_path) as basemap_src:
    basemap_crs = basemap_src.crs
    basemap_transform = basemap_src.transform
    basemap_width = basemap_src.width
    basemap_height = basemap_src.height

print(f"Basemap CRS: {basemap_crs}")
print(f"Basemap dimensions: {basemap_width}x{basemap_height}")

# Convert all GCPs to pixel coordinates
gcp_pixel_coords = {}
for gcp in gcps:
    pixel_coords = gcp_to_pixel_coords(gcp['lat'], gcp['lon'], basemap_path)
    if pixel_coords:
        gcp_pixel_coords[gcp['id']] = {
            'gcp': gcp,
            'pixel_col': pixel_coords[0],
            'pixel_row': pixel_coords[1]
        }
    else:
        print(f"⚠️  GCP {gcp['id']} is outside basemap bounds")

print(f"\n✓ Found {len(gcp_pixel_coords)} GCPs within basemap bounds")
print(f"\nFirst few GCP pixel coordinates:")
for gcp_id, coords in list(gcp_pixel_coords.items())[:3]:
    print(f"  {gcp_id}: col={coords['pixel_col']}, row={coords['pixel_row']}")

## Step 4: Extract Patches from Basemap

In [ ]:
# Extract patches from basemap centered on GCPs
def extract_patch(raster_path: Path, center_col: int, center_row: int, patch_size: int) -> Optional[np.ndarray]:
    """
    Extract a patch from raster centered on given pixel coordinates.
    
    Args:
        raster_path: Path to raster file
        center_col: Center column (x)
        center_row: Center row (y)
        patch_size: Size of patch (must be odd, e.g., 29, 39, 49)
    
    Returns:
        Patch array (H, W, C) or None if out of bounds
    """
    half_size = patch_size // 2
    
    with rasterio.open(raster_path) as src:
        # Calculate bounds
        col_start = max(0, center_col - half_size)
        col_end = min(src.width, center_col + half_size + 1)
        row_start = max(0, center_row - half_size)
        row_end = min(src.height, center_row + half_size + 1)
        
        # Check if patch would be out of bounds
        if col_end - col_start < patch_size or row_end - row_start < patch_size:
            return None
        
        # Read patch
        patch = src.read(
            window=rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)
        )
        
        # Transpose to (H, W, C) format
        if len(patch.shape) == 3:
            patch = np.transpose(patch, (1, 2, 0))
        
        # If single band, convert to 3-channel grayscale
        if len(patch.shape) == 2:
            patch = np.stack([patch, patch, patch], axis=-1)
        
        return patch

# Extract patches for different patch sizes
patch_sizes = [29, 39, 49, 59]  # Try different sizes
basemap_patches = {}

for patch_size in patch_sizes:
    basemap_patches[patch_size] = {}
    
    for gcp_id, coords in gcp_pixel_coords.items():
        patch = extract_patch(
            basemap_path,
            coords['pixel_col'],
            coords['pixel_row'],
            patch_size
        )
        
        if patch is not None:
            basemap_patches[patch_size][gcp_id] = patch
            
            # Save patch as image for visualization
            patch_path = patches_dir / f"basemap_{gcp_id}_{patch_size}x{patch_size}.png"
            plt.imsave(patch_path, patch.astype(np.uint8))
    
    print(f"✓ Extracted {len(basemap_patches[patch_size])} patches of size {patch_size}x{patch_size}")

print(f"\n✓ Patch extraction complete!")

## Step 5: Reproject Orthomosaics to Match Basemap CRS

In [6]:
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_bounds
from rasterio.transform import from_bounds
from rasterio.enums import Resampling as RasterioResampling
from affine import Affine

# Reproject orthos to match basemap CRS and resolution
def reproject_ortho_to_basemap(ortho_path: Path, basemap_path: Path, output_path: Path) -> Path:
    """
    Reproject orthomosaic to match basemap CRS and bounds.
    Uses manual transform construction to avoid CPLE_AppDefinedError.
    """
    if output_path.exists():
        print(f"  ✓ Already reprojected: {output_path}")
        return output_path
    
    with rasterio.open(basemap_path) as basemap_src:
        target_crs = basemap_src.crs
        target_bounds = basemap_src.bounds
        target_transform = basemap_src.transform
        target_width = basemap_src.width
        target_height = basemap_src.height
    
    with rasterio.open(ortho_path) as ortho_src:
        source_crs = ortho_src.crs
        source_bounds = ortho_src.bounds
        
        if source_crs == target_crs:
            print(f"  ✓ Already in target CRS")
            import shutil
            shutil.copy(ortho_path, output_path)
            return output_path
        
        # Transform source bounds to target CRS
        print(f"  Transforming source bounds to target CRS...")
        src_bounds_target_crs = transform_bounds(
            source_crs, target_crs,
            source_bounds.left, source_bounds.bottom,
            source_bounds.right, source_bounds.top
        )
        
        print(f"  Source bounds in target CRS: {src_bounds_target_crs}")
        
        # Get target pixel size
        target_pixel_size_x = abs(target_transform[0])
        target_pixel_size_y = abs(target_transform[4])
        
        # Use intersection of bounds
        output_left = max(src_bounds_target_crs[0], target_bounds.left)
        output_bottom = max(src_bounds_target_crs[1], target_bounds.bottom)
        output_right = min(src_bounds_target_crs[2], target_bounds.right)
        output_top = min(src_bounds_target_crs[3], target_bounds.top)
        
        print(f"  Output bounds (intersection): left={output_left:.2f}, bottom={output_bottom:.2f}, right={output_right:.2f}, top={output_top:.2f}")
        
        # Validate bounds
        if output_right <= output_left or output_top <= output_bottom:
            raise ValueError(f"Invalid output bounds: width={output_right-output_left}, height={output_top-output_bottom}")
        
        # Calculate dimensions using target pixel size
        width = int((output_right - output_left) / target_pixel_size_x)
        height = int((output_top - output_bottom) / target_pixel_size_y)
        
        # Validate dimensions
        if width <= 0 or height <= 0:
            raise ValueError(f"Invalid dimensions: width={width}, height={height}")
        
        # Create transform for output
        transform = Affine.translation(output_left, output_top) * Affine.scale(target_pixel_size_x, -target_pixel_size_y)
        
        print(f"  ✓ Transform calculated: {width}x{height} pixels")
        
        # Read source data
        source_data = ortho_src.read()
        source_count = ortho_src.count
        
        # Reproject
        reprojected_data = np.zeros((source_count, height, width), dtype=source_data.dtype)
        
        for band_idx in range(1, source_count + 1):
            reproject(
                source=rasterio.band(ortho_src, band_idx),
                destination=reprojected_data[band_idx - 1],
                src_transform=ortho_src.transform,
                src_crs=source_crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )
        
        # Save
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=source_count,
            dtype=reprojected_data.dtype,
            crs=target_crs,
            transform=transform,
            compress='lzw',
            BIGTIFF='YES',
            tiled=True,
            blockxsize=512,
            blockysize=512
        ) as dst:
            dst.write(reprojected_data)
    
    return output_path

# Check for existing reprojected files from test_matching notebook
existing_reprojected_dir = output_dir / "test_matching" / "reprojected"
reprojected_dir = gcp_matching_dir / "reprojected"
reprojected_dir.mkdir(exist_ok=True)

ortho_paths = {
    'no_gcps': ortho_no_gcps_path,
    'with_gcps': ortho_with_gcps_path
}

reprojected_paths = {}
for ortho_name, ortho_path in ortho_paths.items():
    if not ortho_path.exists():
        print(f"⚠️  Ortho not found: {ortho_path}")
        continue
    
    # Check for existing reprojected file from test_matching
    existing_reprojected = existing_reprojected_dir / f"{ortho_name}_reprojected.tif"
    if existing_reprojected.exists():
        print(f"\nFound existing reprojected file: {existing_reprojected}")
        # Copy to our directory
        import shutil
        reprojected_path = reprojected_dir / f"{ortho_name}_reprojected.tif"
        if not reprojected_path.exists():
            shutil.copy(existing_reprojected, reprojected_path)
            print(f"  ✓ Copied to: {reprojected_path}")
        else:
            print(f"  ✓ Already exists: {reprojected_path}")
        reprojected_paths[ortho_name] = reprojected_path
        continue
    
    # Otherwise, reproject
    print(f"\nReprojecting {ortho_name}...")
    reprojected_path = reproject_ortho_to_basemap(
        ortho_path,
        basemap_path,
        reprojected_dir / f"{ortho_name}_reprojected.tif"
    )
    reprojected_paths[ortho_name] = reprojected_path

print(f"\n✓ Reprojection complete!")


Reprojecting no_gcps...


CPLE_AppDefinedError: Too many points (10201 out of 10201) failed to transform, unable to compute output bounds.

## Step 6: Find GCP Patches in Orthomosaics Using Template Matching

In [7]:
# Find GCP patches in orthomosaics using template matching
def find_patch_in_ortho(
    template_patch: np.ndarray,
    ortho_path: Path,
    search_center_col: int,
    search_center_row: int,
    search_radius: int = 500  # Search within this radius (pixels)
) -> Optional[Tuple[int, int, float]]:
    """
    Find template patch in orthomosaic using template matching.
    
    Returns:
        (col, row, confidence) or None if not found
    """
    # Convert template to grayscale if needed
    if len(template_patch.shape) == 3:
        template_gray = cv2.cvtColor(template_patch.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    else:
        template_gray = template_patch.astype(np.uint8)
    
    with rasterio.open(ortho_path) as ortho_src:
        # Define search window
        search_col_start = max(0, search_center_col - search_radius)
        search_col_end = min(ortho_src.width, search_center_col + search_radius)
        search_row_start = max(0, search_center_row - search_radius)
        search_row_end = min(ortho_src.height, search_center_row + search_radius)
        
        # Read search region
        search_window = rasterio.windows.Window(
            search_col_start,
            search_row_start,
            search_col_end - search_col_start,
            search_row_end - search_row_start
        )
        
        search_region = ortho_src.read(window=search_window)
        
        # Convert to (H, W, C) and then grayscale
        if len(search_region.shape) == 3:
            search_region = np.transpose(search_region, (1, 2, 0))
            if search_region.shape[2] == 1:
                search_gray = search_region[:, :, 0]
            else:
                search_gray = cv2.cvtColor(search_region.astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            search_gray = search_region
        
        # Normalize to uint8
        if search_gray.dtype != np.uint8:
            search_min = search_gray.min()
            search_max = search_gray.max()
            if search_max > search_min:
                search_gray = ((search_gray - search_min) / (search_max - search_min) * 255).astype(np.uint8)
            else:
                search_gray = np.zeros_like(search_gray, dtype=np.uint8)
        
        # Template matching
        result = cv2.matchTemplate(search_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        
        # Find best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Convert back to global coordinates
        match_col = search_col_start + max_loc[0] + template_gray.shape[1] // 2
        match_row = search_row_start + max_loc[1] + template_gray.shape[0] // 2
        
        # Return if confidence is high enough
        if max_val > 0.5:  # Threshold for match confidence
            return (match_col, match_row, float(max_val))
        else:
            return None

# Find GCPs in each orthomosaic
# Create directory for matching patches
matching_patches_dir = gcp_matching_dir / "matching_patches"
matching_patches_dir.mkdir(exist_ok=True)

matching_results = {}

for ortho_name, reprojected_path in reprojected_paths.items():
    print(f"\n{'='*60}")
    print(f"Finding GCPs in {ortho_name} orthomosaic")
    print(f"{'='*60}")
    
    matching_results[ortho_name] = {}
    
                # Convert GCP UTM coordinates to pixel coordinates in THIS ortho
                # The ortho may have different dimensions/transform than basemap
    with rasterio.open(reprojected_path) as ortho_src:
        ortho_transform = ortho_src.transform
    
    # Try different patch sizes
    best_patch_size = None
    best_matches = 0
    
    for patch_size in patch_sizes:
        matches_found = 0
        
        for gcp_id, coords in gcp_pixel_coords.items():
            if gcp_id not in basemap_patches[patch_size]:
                continue
            
            template = basemap_patches[patch_size][gcp_id]
            
            # Convert GCP UTM coordinates to pixel coordinates in THIS ortho
            # The ortho may have different dimensions/transform than basemap
            gcp_utm_x = coords.get('utm_x') or coords.get('x_utm')
            gcp_utm_y = coords.get('utm_y') or coords.get('y_utm')
            
            if gcp_utm_x is not None and gcp_utm_y is not None:
                # Convert UTM to pixel coordinates using ortho's transform
                expected_col, expected_row = ~ortho_transform * (gcp_utm_x, gcp_utm_y)
                expected_col = int(expected_col)
                expected_row = int(expected_row)
            else:
                # Fallback: use pixel coordinates from basemap (less accurate)
                expected_col = coords['pixel_col']
                expected_row = coords['pixel_row']
                
                if gcp_utm_x is not None and gcp_utm_y is not None:
                    # Convert UTM to pixel coordinates using ortho's transform
                    expected_col, expected_row = ~ortho_transform * (gcp_utm_x, gcp_utm_y)
                    expected_col = int(expected_col)
                    expected_row = int(expected_row)
                else:
                    # Fallback: use pixel coordinates from basemap (less accurate)
                    expected_row = coords['pixel_row']
            
            # Search for patch
            match = find_patch_in_ortho(
                template,
                reprojected_path,
                expected_col,
                expected_row,
                search_radius=500
            )
            
            if match:
                match_col, match_row, confidence = match
                matches_found += 1
                
                if gcp_id not in matching_results[ortho_name]:
                    matching_results[ortho_name][gcp_id] = {}
                
                matching_results[ortho_name][gcp_id][patch_size] = {
                    'expected_col': expected_col,
                    'expected_row': expected_row,
                    'matched_col': match_col,
                    'matched_row': match_row,
                    'offset_col': match_col - expected_col,
                    'offset_row': match_row - expected_row,
                    'confidence': confidence
                }
        
        print(f"  Patch size {patch_size}x{patch_size}: {matches_found}/{len(gcp_pixel_coords)} matches")
        
        if matches_found > best_matches:
            best_matches = matches_found
            best_patch_size = patch_size
    
    print(f"\n  ✓ Best patch size: {best_patch_size}x{best_patch_size} ({best_matches} matches)")

print(f"\n✓ Patch matching complete!")

# Create comprehensive visualization
print(f"\nCreating visualization of GCP matches...")

def create_gcp_matching_visualization(
    basemap_path: Path,
    ortho_paths: Dict[str, Path],
    gcp_pixel_coords: Dict,
    matching_results: Dict,
    output_path: Path,
    max_dimension: int = 4000
):
    """
    Create visualization showing basemap with GCPs and orthos with matched patches.
    """
    # Load basemap
    with rasterio.open(basemap_path) as src:
        basemap_data = src.read()
        basemap_transform = src.transform
        
        # Convert to (H, W, C)
        if len(basemap_data.shape) == 3:
            basemap_img = np.transpose(basemap_data, (1, 2, 0))
            if basemap_img.shape[2] == 1:
                basemap_img = np.stack([basemap_img[:, :, 0]] * 3, axis=-1)
            elif basemap_img.shape[2] == 4:
                basemap_img = basemap_img[:, :, :3]  # Take RGB
        else:
            basemap_img = np.stack([basemap_data] * 3, axis=-1)
        
        # Normalize to uint8
        if basemap_img.dtype != np.uint8:
            basemap_min = basemap_img.min()
            basemap_max = basemap_img.max()
            if basemap_max > basemap_min:
                basemap_img = ((basemap_img - basemap_min) / (basemap_max - basemap_min) * 255).astype(np.uint8)
            else:
                basemap_img = np.zeros_like(basemap_img, dtype=np.uint8)
    
    # Downsample if too large
    h, w = basemap_img.shape[:2]
    if max(h, w) > max_dimension:
        scale = max_dimension / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        basemap_img = cv2.resize(basemap_img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        scale_factor = scale
    else:
        scale_factor = 1.0
    
    # Load orthos and create panels
    num_orthos = len(ortho_paths)
    fig, axes = plt.subplots(1, num_orthos + 1, figsize=(8 * (num_orthos + 1), 8))
    
    # Basemap panel (left)
    ax = axes[0]
    basemap_display = basemap_img.copy()
    
    # Draw GCP positions on basemap
    for gcp_id, coords in gcp_pixel_coords.items():
        # Scale coordinates
        col = int(coords['pixel_col'] * scale_factor)
        row = int(coords['pixel_row'] * scale_factor)
        
        if 0 <= row < basemap_display.shape[0] and 0 <= col < basemap_display.shape[1]:
            # Draw red circle
            cv2.circle(basemap_display, (col, row), 10, (255, 0, 0), 3)
    
    ax.imshow(basemap_display)
    ax.set_title('Basemap with GCP Locations', fontsize=14, fontweight='bold')
    
    # Add GCP labels
    for gcp_id, coords in gcp_pixel_coords.items():
        col = int(coords['pixel_col'] * scale_factor)
        row = int(coords['pixel_row'] * scale_factor)
        if 0 <= row < basemap_display.shape[0] and 0 <= col < basemap_display.shape[1]:
            ax.text(col, row - 15, gcp_id, color='red', fontsize=8, fontweight='bold',
                   ha='center', va='bottom')
    
    ax.axis('off')
    
    # Ortho panels (right)
    for ortho_idx, (ortho_name, ortho_path) in enumerate(ortho_paths.items(), 1):
        ax = axes[ortho_idx]
        
        # Load ortho
        with rasterio.open(ortho_path) as src:
            ortho_data = src.read()
            
            # Convert to (H, W, C)
            if len(ortho_data.shape) == 3:
                ortho_img = np.transpose(ortho_data, (1, 2, 0))
                if ortho_img.shape[2] == 1:
                    ortho_img = np.stack([ortho_img[:, :, 0]] * 3, axis=-1)
                elif ortho_img.shape[2] == 4:
                    ortho_img = ortho_img[:, :, :3]
            else:
                ortho_img = np.stack([ortho_data] * 3, axis=-1)
            
            # Normalize
            if ortho_img.dtype != np.uint8:
                ortho_min = ortho_img.min()
                ortho_max = ortho_img.max()
                if ortho_max > ortho_min:
                    ortho_img = ((ortho_img - ortho_min) / (ortho_max - ortho_min) * 255).astype(np.uint8)
                else:
                    ortho_img = np.zeros_like(ortho_img, dtype=np.uint8)
        
        # Downsample if too large
        h, w = ortho_img.shape[:2]
        if max(h, w) > max_dimension:
            scale = max_dimension / max(h, w)
            new_h, new_w = int(h * scale), int(w * scale)
            ortho_img = cv2.resize(ortho_img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            ortho_scale = scale
        else:
            ortho_scale = 1.0
        
        ortho_display = ortho_img.copy()
        
        # Draw matched patch centers
        if ortho_name in matching_results:
            for gcp_id, match_data in matching_results[ortho_name].items():
                # Get best patch size match
                best_patch_size = max(match_data.keys()) if match_data else None
                if best_patch_size:
                    match = match_data[best_patch_size]
                    matched_col = int(match['matched_col'] * ortho_scale)
                    matched_row = int(match['matched_row'] * ortho_scale)
                    
                    if 0 <= matched_row < ortho_display.shape[0] and 0 <= matched_col < ortho_display.shape[1]:
                        # Draw yellow circle
                        cv2.circle(ortho_display, (matched_col, matched_row), 10, (255, 255, 0), 3)
        
        ax.imshow(ortho_display)
        ax.set_title(f'{ortho_name.replace("_", " ").title()} with Matched Patches', fontsize=14, fontweight='bold')
        
        # Add labels
        if ortho_name in matching_results:
            for gcp_id, match_data in matching_results[ortho_name].items():
                best_patch_size = max(match_data.keys()) if match_data else None
                if best_patch_size:
                    match = match_data[best_patch_size]
                    matched_col = int(match['matched_col'] * ortho_scale)
                    matched_row = int(match['matched_row'] * ortho_scale)
                    if 0 <= matched_row < ortho_display.shape[0] and 0 <= matched_col < ortho_display.shape[1]:
                        ax.text(matched_col, matched_row - 15, gcp_id, color='yellow', fontsize=8, fontweight='bold',
                               ha='center', va='bottom')
        
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', format='PNG')
    plt.close()
    
    print(f"✓ Visualization saved: {output_path}")

# Create visualization for each ortho
for ortho_name in reprojected_paths.keys():
    if ortho_name not in matching_results:
        continue
    
    vis_path = matches_dir / f"gcp_matching_visualization_{ortho_name}.png"

    # Check if visualization already exists
    if vis_path.exists():
        print(f"  ✓ Visualization already exists: {vis_path}")
        print(f"  Skipping visualization creation...")
        continue

    
    create_gcp_matching_visualization(
        basemap_path,
        {ortho_name: reprojected_paths[ortho_name]},
        gcp_pixel_coords,
        matching_results,
        vis_path,
        max_dimension=4000
    )



Finding GCPs in no_gcps orthomosaic
  Patch size 29x29: 22/23 matches
  Patch size 39x39: 21/23 matches
  Patch size 49x49: 21/23 matches
  Patch size 59x59: 21/23 matches

  ✓ Best patch size: 29x29 (22 matches)

Finding GCPs in with_gcps orthomosaic
  Patch size 29x29: 22/23 matches
  Patch size 39x39: 22/23 matches
  Patch size 49x49: 21/23 matches
  Patch size 59x59: 21/23 matches

  ✓ Best patch size: 29x29 (22 matches)

✓ Patch matching complete!

Creating visualization of GCP matches...
✓ Visualization saved: outputs/gcp_matching/matches/gcp_matching_visualization_no_gcps.png
✓ Visualization saved: outputs/gcp_matching/matches/gcp_matching_visualization_with_gcps.png


## Step 7: Compute 2D Shift or Affine Transformation

In [8]:
# Check for required variables and set defaults if needed
try:
    _ = output_dir
except NameError:
    from pathlib import Path
    output_dir = Path("outputs")
    print(f"output_dir not defined, using default: {output_dir}")

try:
    _ = matches_dir
except NameError:
    try:
        _ = gcp_matching_dir
    except NameError:
        gcp_matching_dir = output_dir / "gcp_matching"
    matches_dir = gcp_matching_dir / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)
    print(f"matches_dir not defined, using default: {matches_dir}")

from scipy.spatial.distance import cdist
import numpy as np
from typing import Tuple, Dict

# Compute transformation from matches

def remove_outliers_ransac(src_points: np.ndarray, dst_points: np.ndarray, threshold: float = 50.0, min_samples: int = 3) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Remove outliers using RANSAC-based approach.
    
    Returns:
        (inlier_src, inlier_dst, inlier_mask)
    """
    if len(src_points) < min_samples:
        # Return original arrays with all True mask
        mask = np.ones(len(src_points), dtype=bool)
        return src_points, dst_points, mask
        return src_points, dst_points, np.ones(len(src_points), dtype=bool)
    
    # Compute initial shift (mean offset)
    offsets = dst_points - src_points
    mean_offset = np.mean(offsets, axis=0)
    
    # Compute distances from mean offset
    distances = np.linalg.norm(offsets - mean_offset, axis=1)
    
    # Use IQR method to identify outliers
    q1 = np.percentile(distances, 25)
    q3 = np.percentile(distances, 75)
    iqr = q3 - q1
    
    # Outlier threshold: Q3 + 1.5 * IQR
    outlier_threshold = q3 + 1.5 * iqr
    
    # Also use absolute threshold
    inlier_mask = (distances <= outlier_threshold) & (distances <= threshold)
    
    # Ensure we have at least min_samples inliers
    if np.sum(inlier_mask) < min_samples:
        # Keep the min_samples points closest to the mean
        # Keep the min_samples points closest to the mean
        sorted_indices = np.argsort(distances)
        inlier_mask = np.zeros(len(src_points), dtype=bool)
        inlier_mask[sorted_indices[:min_samples]] = True
        inlier_mask = np.zeros(len(src_points), dtype=bool)
        inlier_mask[sorted_indices[:min_samples]] = True
    
    # Ensure inlier_mask is a proper boolean array
    inlier_mask = np.asarray(inlier_mask, dtype=bool)
    
    # Return filtered points
    return src_points[inlier_mask], dst_points[inlier_mask], inlier_mask

def compute_transformation(matches: Dict, use_affine: bool = False) -> Dict:
    """
    Compute 2D shift or affine transformation from GCP matches.
    
    Args:
        matches: Dictionary with GCP matches
        use_affine: If True, compute affine transformation; otherwise 2D shift
    
    Returns:
        Dictionary with transformation parameters
    """
    # Collect source and destination points
    src_points = []
    dst_points = []
    
    for gcp_id, match_data in matches.items():
        # Use the best patch size match
        best_patch_size = max(match_data.keys())
        match = match_data[best_patch_size]
        
        src_points.append([match['expected_col'], match['expected_row']])
        dst_points.append([match['matched_col'], match['matched_row']])
    
    src_points = np.array(src_points, dtype=np.float32)

    dst_points = np.array(dst_points, dtype=np.float32)

    # Remove outliers
    src_points, dst_points, inlier_mask = remove_outliers_ransac(src_points, dst_points, threshold=100.0, min_samples=3)
    
    num_outliers = len(src_points) - np.sum(inlier_mask) if 'inlier_mask' in locals() else 0
    num_outliers = len(src_points) - np.sum(inlier_mask) if inlier_mask is not None else 0
    if num_outliers > 0:
        print(f"  Removed {num_outliers} outlier(s)")
    dst_points = np.array(dst_points, dtype=np.float32)
    
    if len(src_points) < min_samples:
        # Return original arrays with all True mask
        mask = np.ones(len(src_points), dtype=bool)
        return src_points, dst_points, mask
        return {'type': 'insufficient_points', 'error': 'Need at least 2 matches'}
    
    if use_affine and len(src_points) >= 3:
        # Compute affine transformation (6 parameters)
        # Requires at least 3 points
        transform_matrix = cv2.getAffineTransform(
            src_points[:3],
            dst_points[:3]
        )
        
        # Apply to all points to compute error
        ones = np.ones((len(src_points), 1))
        src_homogeneous = np.hstack([src_points, ones])
        transformed = (transform_matrix @ src_homogeneous.T).T
        
        errors = dst_points - transformed
        rmse = float(np.sqrt(np.mean(errors**2)))
        
        return {
            'type': 'affine',
            'matrix': transform_matrix.tolist(),
            'rmse': rmse,
            'num_points': len(src_points)
        }
    else:
        # Compute 2D shift (mean offset)
        offsets = dst_points - src_points
        shift_x = float(np.mean(offsets[:, 0]))
        shift_y = float(np.mean(offsets[:, 1]))
        
        # Compute RMSE
        errors = offsets - np.array([shift_x, shift_y])
        rmse = float(np.sqrt(np.mean(errors**2)))
        
        return {
            'type': 'shift',
            'shift_x': shift_x,
            'shift_y': shift_y,
            'rmse': rmse,
            'num_points': len(src_points)
        }

# Compute transformations for each ortho
transformations = {}


# Check if matching_results is defined, load from file if not
try:
    _ = matching_results
    print("✓ matching_results found in memory")
except NameError:
    print("matching_results not in memory, attempting to load from file...")
    try:
        matches_json = matches_dir / "matching_results.json"
        if matches_json.exists():
            with open(matches_json, 'r') as f:
                matching_results = json.load(f)
            print(f"✓ Loaded matching_results from {matches_json}")
        else:
            raise FileNotFoundError(f"matching_results.json not found at {matches_json}")
    except Exception as e:
        print(f"❌ Could not load matching_results: {e}")
        print("Please run Step 6 (patch matching) first.")
        raise

for ortho_name in matching_results.keys():

    # Print match pixel locations for verification
    print(f"\nMatch pixel locations:")
    for gcp_id, match_data in matching_results[ortho_name].items():
        best_patch_size = max(match_data.keys()) if match_data else None
        if best_patch_size:
            match = match_data[best_patch_size]
            print(f"  {gcp_id}: Expected=({match['expected_col']:.1f}, {match['expected_row']:.1f}), ")
            print(f"         Matched=({match['matched_col']:.1f}, {match['matched_row']:.1f}), ")
            print(f"         Offset=({match['offset_col']:.1f}, {match['offset_row']:.1f}) px")
    print(f"\n{'='*60}")
    print(f"Computing transformation for {ortho_name}")
    print(f"{'='*60}")
    
    # Try 2D shift first
    shift_result = compute_transformation(matching_results[ortho_name], use_affine=False)
    print(f"\n2D Shift:")
    shift_x_val = shift_result.get('shift_x', 'N/A')
    shift_x_str = f"{shift_x_val:.2f}" if isinstance(shift_x_val, (int, float)) else str(shift_x_val)
    print(f"  Shift X: {shift_x_str} px")
    shift_y_val = shift_result.get('shift_y', 'N/A')
    shift_y_str = f"{shift_y_val:.2f}" if isinstance(shift_y_val, (int, float)) else str(shift_y_val)
    print(f"  Shift Y: {shift_y_str} px")
    rmse_val = shift_result.get('rmse', 'N/A')
    rmse_str = f"{rmse_val:.2f}" if isinstance(rmse_val, (int, float)) else str(rmse_val)
    print(f"  RMSE: {rmse_str} px")
    print(f"  Points: {shift_result.get('num_points', 0)}")
    
    # Try affine if we have enough points
    # Try affine if we have enough points
    if len(matching_results[ortho_name]) >= 3:
        affine_result = compute_transformation(matching_results[ortho_name], use_affine=True)
        print(f"\nAffine Transformation:")
        affine_rmse_val = affine_result.get('rmse', 'N/A')

        affine_rmse_str = f"{affine_rmse_val:.2f}" if isinstance(affine_rmse_val, (int, float)) else str(affine_rmse_val)

        print(f"  RMSE: {affine_rmse_str} px")
        print(f"  Points: {affine_result.get('num_points', 0)}")

        # Use the one with lower RMSE
        if affine_result.get('rmse', float('inf')) < shift_result.get('rmse', float('inf')):
            transformations[ortho_name] = affine_result
            print(f"  ✓ Using affine transformation (lower RMSE)")
        else:
            transformations[ortho_name] = shift_result
            print(f"  ✓ Using 2D shift (lower RMSE)")
    else:
        transformations[ortho_name] = shift_result
        print(f"  ✓ Using 2D shift (insufficient points for affine)")

transformations_file = matches_dir / "transformations.json"

with open(transformations_file, 'w') as f:
    json.dump(transformations, f, indent=2)

print(f"\n✓ Transformations saved to: {transformations_file}")



Computing transformation for no_gcps

2D Shift:
  Shift X: -67.23 px
  Shift Y: -7.91 px
  RMSE: 286.35 px
  Points: 22

Affine Transformation:
  RMSE: 1291.77 px
  Points: 22
  ✓ Using 2D shift (lower RMSE)

Computing transformation for with_gcps

2D Shift:
  Shift X: -124.45 px
  Shift Y: -37.95 px
  RMSE: 267.75 px
  Points: 22

Affine Transformation:
  RMSE: 1889.43 px
  Points: 22
  ✓ Using 2D shift (lower RMSE)

✓ Transformations saved to: outputs/gcp_matching/matches/transformations.json


## Step 8: Apply Transformation and Register Orthomosaics

In [ ]:
# Apply transformation to orthomosaic
def apply_transformation(
    ortho_path: Path,
    transformation: Dict,
    output_path: Path,
    basemap_path: Path
) -> Path:
    """
    Apply transformation to register orthomosaic to basemap.
    """
    with rasterio.open(basemap_path) as basemap_src:
        target_width = basemap_src.width
        target_height = basemap_src.height
        target_transform = basemap_src.transform
        target_crs = basemap_src.crs
    
    with rasterio.open(ortho_path) as ortho_src:
        source_data = ortho_src.read()
        source_count = ortho_src.count
        
        # Apply transformation
        if transformation['type'] == 'shift':
            # Apply 2D shift using scipy
            shift_x = transformation['shift_x']
            shift_y = transformation['shift_y']
            
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
            
            for band_idx in range(source_count):
                shifted = ndimage.shift(
                    source_data[band_idx],
                    (shift_y, shift_x),
                    mode='constant',
                    cval=0,
                    order=1
                )
                
                # Crop or pad to match target dimensions
                if shifted.shape[0] > target_height:
                    shifted = shifted[:target_height, :]
                elif shifted.shape[0] < target_height:
                    padded = np.zeros((target_height, shifted.shape[1]), dtype=shifted.dtype)
                    padded[:shifted.shape[0], :] = shifted
                    shifted = padded
                
                if shifted.shape[1] > target_width:
                    shifted = shifted[:, :target_width]
                elif shifted.shape[1] < target_width:
                    padded = np.zeros((target_height, target_width), dtype=shifted.dtype)
                    padded[:, :shifted.shape[1]] = shifted
                    shifted = padded
                
                registered_data[band_idx] = shifted
        
        elif transformation['type'] == 'affine':
            # Apply affine transformation
            transform_matrix = np.array(transformation['matrix'], dtype=np.float32)
            
            registered_data = np.zeros((source_count, target_height, target_width), dtype=source_data.dtype)
            
            # Convert to (H, W, C) for OpenCV
            if source_count == 1:
                img = source_data[0]
            else:
                img = np.transpose(source_data, (1, 2, 0))
            
            # Apply affine transform
            registered_img = cv2.warpAffine(
                img.astype(np.uint8),
                transform_matrix,
                (target_width, target_height),
                flags=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=0
            )
            
            # Convert back to (C, H, W)
            if source_count == 1:
                registered_data[0] = registered_img
            else:
                registered_data = np.transpose(registered_img, (2, 0, 1))
        
        # Save registered orthomosaic
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=target_height,
            width=target_width,
            count=source_count,
            dtype=registered_data.dtype,
            crs=target_crs,
            transform=target_transform,
            compress='lzw',
            BIGTIFF='YES',
            tiled=True
        ) as dst:
            dst.write(registered_data)
    
    return output_path

# Register orthos
registered_paths = {}

for ortho_name, transformation in transformations.items():
    if 'error' in transformation:
        print(f"⚠️  Skipping {ortho_name}: {transformation['error']}")
        continue
    
    print(f"\nRegistering {ortho_name}...")
    
    registered_path = apply_transformation(
        reprojected_paths[ortho_name],
        transformation,
        registered_dir / f"{ortho_name}_registered.tif",
        basemap_path
    )
    
    registered_paths[ortho_name] = registered_path
    print(f"  ✓ Saved: {registered_path}")

print(f"\n✓ Registration complete!")


Registering no_gcps...


## Step 9: Evaluate Accuracy Improvement

In [ ]:
# Compare registered orthos to basemap
def evaluate_accuracy(ortho_path: Path, basemap_path: Path, gcps: List[Dict]) -> Dict:
    """
    Evaluate accuracy by comparing pixel values at GCP locations.
    """
    with rasterio.open(basemap_path) as basemap_src:
        basemap_data = basemap_src.read()
    
    with rasterio.open(ortho_path) as ortho_src:
        ortho_data = ortho_src.read()
    
    errors = []
    
    for gcp in gcps:
        pixel_coords = gcp_to_pixel_coords(gcp['lat'], gcp['lon'], basemap_path)
        if not pixel_coords:
            continue
        
        col, row = pixel_coords
        
        if 0 <= row < basemap_data.shape[1] and 0 <= col < basemap_data.shape[2]:
            basemap_pixel = basemap_data[:, row, col]
            
            if 0 <= row < ortho_data.shape[1] and 0 <= col < ortho_data.shape[2]:
                ortho_pixel = ortho_data[:, row, col]
                
                # Compute error (Euclidean distance in pixel space)
                error = np.sqrt(np.sum((basemap_pixel.astype(float) - ortho_pixel.astype(float))**2))
                errors.append(error)
    
    if errors:
        return {
            'mean_error': float(np.mean(errors)),
            'rmse': float(np.sqrt(np.mean(np.array(errors)**2))),
            'max_error': float(np.max(errors)),
            'num_points': len(errors)
        }
    else:
        return {'error': 'No valid GCPs found'}

# Evaluate accuracy for original and registered orthos
accuracy_results = {}

for ortho_name in reprojected_paths.keys():
    print(f"\n{'='*60}")
    print(f"Accuracy evaluation: {ortho_name}")
    print(f"{'='*60}")
    
    # Original (reprojected but not registered)
    original_accuracy = evaluate_accuracy(
        reprojected_paths[ortho_name],
        basemap_path,
        gcps
    )
    
    print(f"\nOriginal (reprojected):")
    print(f"  Mean error: {original_accuracy.get('mean_error', 'N/A'):.2f}")
    print(f"  RMSE: {original_accuracy.get('rmse', 'N/A'):.2f}")
    
    # Registered
    if ortho_name in registered_paths:
        registered_accuracy = evaluate_accuracy(
            registered_paths[ortho_name],
            basemap_path,
            gcps
        )
        
        print(f"\nRegistered:")
        print(f"  Mean error: {registered_accuracy.get('mean_error', 'N/A'):.2f}")
        print(f"  RMSE: {registered_accuracy.get('rmse', 'N/A'):.2f}")
        
        improvement = ((original_accuracy.get('rmse', 0) - registered_accuracy.get('rmse', 0)) / original_accuracy.get('rmse', 1)) * 100
        print(f"\n  Improvement: {improvement:.1f}%")
        
        accuracy_results[ortho_name] = {
            'original': original_accuracy,
            'registered': registered_accuracy,
            'improvement_percent': improvement
        }
    else:
        accuracy_results[ortho_name] = {
            'original': original_accuracy
        }

# Save results
results_file = gcp_matching_dir / "accuracy_results.json"
with open(results_file, 'w') as f:
    json.dump(accuracy_results, f, indent=2)

print(f"\n✓ Accuracy results saved to: {results_file}")